In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import dspy
import asyncio
import os
from dotenv import load_dotenv
from datasets import load_dataset
import logging

from nano_graphrag._utils import compute_mdhash_id
from nano_graphrag.entity_extraction.dataset import generate_dataset
from nano_graphrag.entity_extraction.optimize import compile_model
from nano_graphrag.entity_extraction.module import EntityRelationshipExtractor

/opt/homebrew/Caskroom/miniconda/base/envs/nano-graphrag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
/opt/homebrew/Caskroom/miniconda/base/envs/nano-graphrag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
WORKING_DIR = "./nano_graphrag_cache_finetune_entity_relationship_dspy"

load_dotenv()

logging.basicConfig(level=logging.WARNING)
logging.getLogger("nano-graphrag").setLevel(logging.DEBUG)

In [4]:
system_prompt = """
    You are a world-class AI system, capable of complex reasoning and reflection. 
    Reason through the query, and then provide your final response. 
    If you detect that you made a mistake in your reasoning at any point, correct yourself.
    Think carefully.
"""
lm = dspy.OpenAI(
    model="deepseek-chat", 
    model_type="chat", 
    api_key=os.environ["DEEPSEEK_API_KEY"], 
    base_url=os.environ["DEEPSEEK_BASE_URL"], 
    system_prompt=system_prompt, 
    temperature=0.3,
    top_p=1.0,
    max_tokens=4096
)
dspy.settings.configure(lm=lm)

In [5]:
os.makedirs(WORKING_DIR, exist_ok=True)
train_len = 20
entity_relationship_dataset_path = os.path.join(WORKING_DIR, "entity_relationship_extraction_news.pkl")
entity_relationship_module_path = os.path.join(WORKING_DIR, "entity_relationship_extraction_news.json")
ds = load_dataset("ashraq/financial-news-articles")
train_data = ds['train'][-train_len:]

/opt/homebrew/Caskroom/miniconda/base/envs/nano-graphrag/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [6]:
train_data['text'][-1]

'May 4 (Reuters) - Wolford AG:\n* FOSUN INDUSTRIAL HOLDINGS LIMITED IS THE NEW MAJORITY SHAREHOLDER, HOLDING APPROX. 50.87% OF THE SHARE CAPITAL Source text for Eikon: Further company coverage: (Gdynia Newsroom)\n '

In [7]:
chunks = {compute_mdhash_id(text, prefix=f"chunk-"): {"content": text} for text in train_data["text"]}
dataset = asyncio.run(generate_dataset(chunks=chunks, filepath=entity_relationship_dataset_path))

DEBUG:nano-graphrag:Entities: 8 | Missed Entities: 5 | Total Entities: 13
DEBUG:nano-graphrag:Entities: 21 | Missed Entities: 8 | Total Entities: 29
DEBUG:nano-graphrag:Entities: 5 | Missed Entities: 4 | Total Entities: 9
DEBUG:nano-graphrag:Relationships: 7 | Missed Relationships: 7 | Total Relationships: 14
DEBUG:nano-graphrag:Relationships: 15 | Missed Relationships: 7 | Total Relationships: 22
DEBUG:nano-graphrag:Relationships: 4 | Missed Relationships: 6 | Total Relationships: 10
DEBUG:nano-graphrag:Entities: 11 | Missed Entities: 2 | Total Entities: 13
DEBUG:nano-graphrag:Entities: 5 | Missed Entities: 3 | Total Entities: 8
DEBUG:nano-graphrag:Entities: 14 | Missed Entities: 5 | Total Entities: 19
DEBUG:nano-graphrag:Entities: 6 | Missed Entities: 2 | Total Entities: 8
DEBUG:nano-graphrag:Relationships: 9 | Missed Relationships: 9 | Total Relationships: 18
DEBUG:nano-graphrag:Entities: 17 | Missed Entities: 15 | Total Entities: 32
DEBUG:nano-graphrag:Relationships: 4 | Missed Rel

In [8]:
dataset[0]

Example({'input_text': 'May 8 (Reuters) - BRT Apartments Corp:\n* BRT APARTMENTS CORP. REPORTS SECOND FISCAL QUARTER RESULTS FOR MARCH 31, 2018\n* Q2 ADJUSTED FFO PER SHARE $0.26 * Q2 FFO PER SHARE $0.37 Source text for Eikon: Further company coverage:\n ', 'entities': Entities(context=[Entity(entity_name='BRT APARTMENTS CORP', entity_type='ORGANIZATION', description='A company that reported its second fiscal quarter results for March 31, 2018.', importance_score=0.9), Entity(entity_name='MARCH 31, 2018', entity_type='DATE', description='The specific date for which BRT Apartments Corp. reported its second fiscal quarter results.', importance_score=0.8), Entity(entity_name='Q2 ADJUSTED FFO PER SHARE', entity_type='TITLE', description='A financial metric indicating the adjusted funds from operations per share for the second quarter.', importance_score=0.7), Entity(entity_name='Q2 FFO PER SHARE', entity_type='TITLE', description='A financial metric indicating the funds from operations per

In [9]:
dataset[0].relationships.context

[Relationship(src_id='BRT APARTMENTS CORP', tgt_id='MARCH 31, 2018', description='BRT Apartments Corp. reported its second fiscal quarter results for March 31, 2018.', weight=0.8),
 Relationship(src_id='BRT APARTMENTS CORP', tgt_id='Q2 ADJUSTED FFO PER SHARE', description='BRT Apartments Corp. reported Q2 adjusted FFO per share as a financial metric.', weight=0.7),
 Relationship(src_id='BRT APARTMENTS CORP', tgt_id='Q2 FFO PER SHARE', description='BRT Apartments Corp. reported Q2 FFO per share as a financial metric.', weight=0.7),
 Relationship(src_id='Q2 ADJUSTED FFO PER SHARE', tgt_id='0.26', description='The Q2 adjusted FFO per share value is $0.26.', weight=0.6),
 Relationship(src_id='Q2 FFO PER SHARE', tgt_id='0.37', description='The Q2 FFO per share value is $0.37.', weight=0.6),
 Relationship(src_id='REUTERS', tgt_id='BRT APARTMENTS CORP', description="Reuters provided information about BRT Apartments Corp.'s financial report.", weight=0.7),
 Relationship(src_id='BRT APARTMENTS 

In [10]:
model = EntityRelationshipExtractor()
model

extractor.predictor = Predict(CombinedExtraction(input_text, entity_types -> entities, relationships
    instructions='Signature for extracting both entities and relationships from input text.'
    input_text = Field(annotation=str required=True json_schema_extra={'desc': 'The text to extract entities and relationships from.', '__dspy_field_type': 'input', 'prefix': 'Input Text:'})
    entity_types = Field(annotation=EntityTypes required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Entity Types:', 'desc': '${entity_types}'})
    entities = Field(annotation=Entities required=True json_schema_extra={'desc': '\n        Format:\n        {\n            "context": [\n                {\n                    "entity_name": "ENTITY NAME",\n                    "entity_type": "ENTITY TYPE",\n                    "description": "Detailed description",\n                    "importance_score": 0.8\n                },\n                ...\n            ]\n        }\n        Each enti

In [11]:
optimized_model = compile_model(
    model=model,
    dataset_path=entity_relationship_dataset_path,
    module_path=entity_relationship_module_path
)
optimized_model

INFO:nano-graphrag:Loaded 20 examples with keys: ['input_text', 'entities', 'relationships']
INFO:nano-graphrag:Evaluating uncompiled DSPy module
DEBUG:nano-graphrag:Entities: 6 | Missed Entities: 2 | Total Entities: 8
DEBUG:nano-graphrag:Relationships: 5 | Missed Relationships: 2 | Total Relationships: 7
DEBUG:nano-graphrag:Entities: 5 | Missed Entities: 4 | Total Entities: 9
DEBUG:nano-graphrag:Entities: 21 | Missed Entities: 8 | Total Entities: 29
DEBUG:nano-graphrag:Entities: 41 | Missed Entities: 20 | Total Entities: 61

DEBUG:nano-graphrag:Entities: 16 | Missed Entities: 10 | Total Entities: 26DEBUG:nano-graphrag:Entities: 5 | Missed Entities: 3 | Total Entities: 8
  0%|          | 0/20 [00:00<?, ?it/s]DEBUG:nano-graphrag:Entities: 8 | Missed Entities: 5 | Total Entities: 13
DEBUG:nano-graphrag:Entities: 14 | Missed Entities: 26 | Total Entities: 40
DEBUG:nano-graphrag:Entities: 11 | Missed Entities: 2 | Total Entities: 13
DEBUG:nano-graphrag:Entities: 8 | Missed Entities: 14 | T

,input_text,example_entities,example_relationships,pred_entities,pred_relationships,relationship_similarity_metric
0,"May 8 (Reuters) - BRT Apartments Corp: * BRT APARTMENTS CORP. REPORTS SECOND FISCAL QUARTER RESULTS FOR MARCH 31, 2018 * Q2 ADJUSTED FFO PER...","context=[Entity(entity_name='BRT APARTMENTS CORP', entity_type='ORGANIZATION', description='A company that reported its second fiscal quarter results for March 31, 2018.', importance_score=0.9), Entity(entity_name='MARCH 31, 2018', entity_type='DATE', description='The specific date...","context=[Relationship(src_id='BRT APARTMENTS CORP', tgt_id='MARCH 31, 2018', description='BRT Apartments Corp. reported its second fiscal quarter results for March 31, 2018.', weight=0.8), Relationship(src_id='BRT APARTMENTS CORP', tgt_id='Q2 ADJUSTED...","context=[Entity(entity_name='BRT APARTMENTS CORP', entity_type='ORGANIZATION', description='A company that reported its second fiscal quarter results for March 31, 2018.', importance_score=0.9), Entity(entity_name='MARCH 31, 2018', entity_type='DATE', description='The specific date...","context=[Relationship(src_id='BRT APARTMENTS CORP', tgt_id='MARCH 31, 2018', description='BRT Apartments Corp. reported its second fiscal quarter results for March 31, 2018.', weight=0.8), Relationship(src_id='BRT APARTMENTS CORP', tgt_id='Q2 ADJUSTED...",✔️ [1.0000001192092896]
1,"May 22, 2018 / 8:59 AM / Updated 18 minutes ago UPDATE 2-European shares rise as Italy recovers and China tariff cut boosts autos Reuters...","context=[Entity(entity_name='MAY 22, 2018', entity_type='DATE', description='The specific date of the event.', importance_score=0.9), Entity(entity_name='LONDON', entity_type='LOCATION', description='The location where the event took place.', importance_score=0.8), Entity(entity_name='REUTERS', entity_type='ORGANIZATION', description='The news...","context=[Relationship(src_id='STOXX 600', tgt_id='CHINA', description='The STOXX 600 index rose due to a tariff cut in China.', weight=0.8), Relationship(src_id='VOLKSWAGEN', tgt_id='CHINA', description=""Volkswagen's stock rose due to a tariff...","context=[Entity(entity_name='MAY 22, 2018', entity_type='DATE', description='The specific date of the event.', importance_score=0.9), Entity(entity_name='LONDON', entity_type='LOCATION', description='The location where the event took place.', importance_score=0.8), Entity(entity_name='REUTERS', entity_type='ORGANIZATION', description='The news...","context=[Relationship(src_id='STOXX 600', tgt_id='CHINA', description='The STOXX 600 index rose due to a tariff cut in China.', weight=0.8), Relationship(src_id='VOLKSWAGEN', tgt_id='CHINA', description=""Volkswagen's stock rose due to a tariff...",✔️ [1.0000001192092896]
2,May 22 (Reuters) - MetLife Inc: * METLIFE ANNOUNCES NEW $1.5 BILLION SHARE REPURCHASE AUTHORIZATION * METLIFE INC - INTENDS TO DIVEST ITS SHARES OF...,"context=[Entity(entity_name='METLIFE', entity_type='ORGANIZATION', description='A major insurance company that announced a new share repurchase authorization and intends to divest its shares of Brighthouse Financial.', importance_score=0.9), Entity(entity_name='BRIGHTHOUSE FINANCIAL',...","context=[Relationship(src_id='METLIFE', tgt_id='BRIGHTHOUSE FINANCIAL', description=""MetLife intends to divest its shares of Brighthouse Financial's common stock."", weight=0.8), Relationship(src_id='METLIFE', tgt_id='2018', description='MetLife expects to complete the divestiture of Brighthouse...","context=[Entity(entity_name='METLIFE', entity_type='ORGANIZATION', description='A major insurance company that announced a new share repurchase authorization and intends to divest its shares of Brighthouse Financial.', importance_score=0.9), Entity(entity_name='BRIGHTHOUSE FINANCIAL',...","context=[Relationship(src_id='METLIFE', tgt_id='BRIGHTHOUSE FINANCIAL', description=""MetLife intends to divest its shares of Brighthouse Financial's common stock."", weight=0.8), Relationship(src_id='METLIFE', tg

INFO:nano-graphrag:Optimizing DSPy module
DEBUG:nano-graphrag:Entities: 21 | Missed Entities: 8 | Total Entities: 29
DEBUG:nano-graphrag:Entities: 6 | Missed Entities: 2 | Total Entities: 8
DEBUG:nano-graphrag:Entities: 5 | Missed Entities: 4 | Total Entities: 9
DEBUG:nano-graphrag:Relationships: 15 | Missed Relationships: 7 | Total Relationships: 22


Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 5 candidate sets.


  0%|          | 0/20 [00:00<?, ?it/s]DEBUG:nano-graphrag:Entities: 41 | Missed Entities: 20 | Total Entities: 61
DEBUG:nano-graphrag:Entities: 16 | Missed Entities: 10 | Total Entities: 26

DEBUG:nano-graphrag:Entities: 8 | Missed Entities: 5 | Total Entities: 13DEBUG:nano-graphrag:Entities: 5 | Missed Entities: 3 | Total Entities: 8
DEBUG:nano-graphrag:Entities: 15 | Missed Entities: 9 | Total Entities: 24
DEBUG:nano-graphrag:Relationships: 5 | Missed Relationships: 2 | Total Relationships: 7
DEBUG:nano-graphrag:Entities: 11 | Missed Entities: 2 | Total Entities: 13
DEBUG:nano-graphrag:Entities: 14 | Missed Entities: 26 | Total Entities: 40
DEBUG:nano-graphrag:Entities: 8 | Missed Entities: 14 | Total Entities: 22


DEBUG:nano-graphrag:Entities: 14 | Missed Entities: 5 | Total Entities: 19
Batches:   0%|          | 0/1 [00:00<?, ?it/s]DEBUG:nano-graphrag:Relationships: 4 | Missed Relationships: 6 | Total Relationships: 10DEBUG:nano-graphrag:Relationships: 31 | Missed Relationships: 2

...

DEBUG:nano-graphrag:Entities: 16 | Missed Entities: 10 | Total Entities: 26
  0%|          | 0/20 [00:00<?, ?it/s]DEBUG:nano-graphrag:Entities: 8 | Missed Entities: 5 | Total Entities: 13
DEBUG:nano-graphrag:Entities: 5 | Missed Entities: 3 | Total Entities: 8
DEBUG:nano-graphrag:Relationships: 5 | Missed Relationships: 2 | Total Relationships: 7
DEBUG:nano-graphrag:Entities: 15 | Missed Entities: 9 | Total Entities: 24
DEBUG:nano-graphrag:Entities: 11 | Missed Entities: 2 | Total Entities: 13
DEBUG:nano-graphrag:Relationships: 15 | Missed Relationships: 7 | Total Relationships: 22

Batches:   0%|          | 0/1 [00:00<?, ?it/s]DEBUG:nano-graphrag:Entities: 14 | Missed Entities: 26 | Total Entities: 40DEBUG:nano-graphrag:Entities: 8 | Missed Entities: 14 | Total Entities: 22
DEBUG:nano-graphrag:Entities: 14 | Missed Entities: 5 | Total Entities: 19


DEBUG:nano-graphrag:Relationships: 4 | Missed Relationships: 6 | Total Relationships: 10
Batches:   0%|          | 0/1 [00:00<?, ?it/s]DE

,input_text,example_entities,example_relationships,pred_entities,pred_relationships,relationship_similarity_metric
0,"May 8 (Reuters) - BRT Apartments Corp: * BRT APARTMENTS CORP. REPORTS SECOND FISCAL QUARTER RESULTS FOR MARCH 31, 2018 * Q2 ADJUSTED FFO PER...","context=[Entity(entity_name='BRT APARTMENTS CORP', entity_type='ORGANIZATION', description='A company that reported its second fiscal quarter results for March 31, 2018.', importance_score=0.9), Entity(entity_name='MARCH 31, 2018', entity_type='DATE', description='The specific date...","context=[Relationship(src_id='BRT APARTMENTS CORP', tgt_id='MARCH 31, 2018', description='BRT Apartments Corp. reported its second fiscal quarter results for March 31, 2018.', weight=0.8), Relationship(src_id='BRT APARTMENTS CORP', tgt_id='Q2 ADJUSTED...","context=[Entity(entity_name='BRT APARTMENTS CORP', entity_type='ORGANIZATION', description='A company that reported its second fiscal quarter results for March 31, 2018.', importance_score=0.9), Entity(entity_name='MARCH 31, 2018', entity_type='DATE', description='The specific date...","context=[Relationship(src_id='BRT APARTMENTS CORP', tgt_id='MARCH 31, 2018', description='BRT Apartments Corp. reported its second fiscal quarter results for March 31, 2018.', weight=0.8), Relationship(src_id='BRT APARTMENTS CORP', tgt_id='Q2 ADJUSTED...",✔️ [1.0000001192092896]
1,"May 22, 2018 / 8:59 AM / Updated 18 minutes ago UPDATE 2-European shares rise as Italy recovers and China tariff cut boosts autos Reuters...","context=[Entity(entity_name='MAY 22, 2018', entity_type='DATE', description='The specific date of the event.', importance_score=0.9), Entity(entity_name='LONDON', entity_type='LOCATION', description='The location where the event took place.', importance_score=0.8), Entity(entity_name='REUTERS', entity_type='ORGANIZATION', description='The news...","context=[Relationship(src_id='STOXX 600', tgt_id='CHINA', description='The STOXX 600 index rose due to a tariff cut in China.', weight=0.8), Relationship(src_id='VOLKSWAGEN', tgt_id='CHINA', description=""Volkswagen's stock rose due to a tariff...","context=[Entity(entity_name='MAY 22, 2018', entity_type='DATE', description='The specific date of the event.', importance_score=0.9), Entity(entity_name='LONDON', entity_type='LOCATION', description='The location where the event took place.', importance_score=0.8), Entity(entity_name='REUTERS', entity_type='ORGANIZATION', description='The news...","context=[Relationship(src_id='STOXX 600', tgt_id='CHINA', description='The STOXX 600 index rose due to a tariff cut in China.', weight=0.8), Relationship(src_id='VOLKSWAGEN', tgt_id='CHINA', description=""Volkswagen's stock rose due to a tariff...",✔️ [1.0000001192092896]
2,May 22 (Reuters) - MetLife Inc: * METLIFE ANNOUNCES NEW $1.5 BILLION SHARE REPURCHASE AUTHORIZATION * METLIFE INC - INTENDS TO DIVEST ITS SHARES OF...,"context=[Entity(entity_name='METLIFE', entity_type='ORGANIZATION', description='A major insurance company that announced a new share repurchase authorization and intends to divest its shares of Brighthouse Financial.', importance_score=0.9), Entity(entity_name='BRIGHTHOUSE FINANCIAL',...","context=[Relationship(src_id='METLIFE', tgt_id='BRIGHTHOUSE FINANCIAL', description=""MetLife intends to divest its shares of Brighthouse Financial's common stock."", weight=0.8), Relationship(src_id='METLIFE', tgt_id='2018', description='MetLife expects to complete the divestiture of Brighthouse...","context=[Entity(entity_name='METLIFE', entity_type='ORGANIZATION', description='A major insurance company that announced a new share repurchase authorization and intends to divest its shares of Brighthouse Financial.', importance_score=0.9), Entity(entity_name='BRIGHTHOUSE FINANCIAL',...","context=[Relationship(src_id='METLIFE', tgt_id='BRIGHTHOUSE FINANCIAL', description=""MetLife intends to divest its shares of Brighthouse Financial's common stock."", weight=0.8), Relationship(src_id='METLIFE', tg

INFO:nano-graphrag:Successfully saved optimized DSPy module to ./nano_graphrag_cache_finetune_entity_relationship_dspy/entity_relationship_extraction_news.json


[('extractor.predictor', Predict(CombinedExtraction(input_text, entity_types -> entities, relationships
    instructions='Signature for extracting both entities and relationships from input text.'
    input_text = Field(annotation=str required=True json_schema_extra={'desc': 'The text to extract entities and relationships from.', '__dspy_field_type': 'input', 'prefix': 'Input Text:'})
    entity_types = Field(annotation=EntityTypes required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Entity Types:', 'desc': '${entity_types}'})
    entities = Field(annotation=Entities required=True json_schema_extra={'desc': '\n        Format:\n        {\n            "context": [\n                {\n                    "entity_name": "ENTITY NAME",\n                    "entity_type": "ENTITY TYPE",\n                    "description": "Detailed description",\n                    "importance_score": 0.8\n                },\n                ...\n            ]\n        }\n        Each e

extractor.predictor = Predict(CombinedExtraction(input_text, entity_types -> entities, relationships
    instructions='Signature for extracting both entities and relationships from input text.'
    input_text = Field(annotation=str required=True json_schema_extra={'desc': 'The text to extract entities and relationships from.', '__dspy_field_type': 'input', 'prefix': 'Input Text:'})
    entity_types = Field(annotation=EntityTypes required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Entity Types:', 'desc': '${entity_types}'})
    entities = Field(annotation=Entities required=True json_schema_extra={'desc': '\n        Format:\n        {\n            "context": [\n                {\n                    "entity_name": "ENTITY NAME",\n                    "entity_type": "ENTITY TYPE",\n                    "description": "Detailed description",\n                    "importance_score": 0.8\n                },\n                ...\n            ]\n        }\n        Each enti